# A 2D grain boundary in Boron Nitride

## 0. Introduction

This notebook demonstrates how to generate a 2D grain boundary in Boron Nitride, following the example in the manuscript:

> **Qiucheng Li, Xiaolong Zou, Mengxi Liu, Jingyu Sun, Yabo Gao, Yue Qi, Xiebo Zhou, Boris I. Yakobson, Yanfeng Zhang, and Zhongfan Liu**
> "Grain Boundary Structures and Electronic Properties of Hexagonal Boron Nitride on Cu(111)"
> *ACS Nano* **2015** 9 (6), 6308-6315
> [DOI: 10.1021/acs.nanolett.5b01852](https://doi.org/10.1021/acs.nanolett.5b01852)

Reproducing the material from Figure 2. c:

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_planar_grain_boundary_2d_boron_nitride/0-figure-from-manuscript.webp" alt="Grain Boundary in Boron Nitride" width="400"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior
For more information on the parameters and algorithm, refer to [Grain Boundary Builder Source](https://github.com/Exabyte-io/made/blob/35b9f318f5d667e0f5af023f3178bc4404317ab0/src/py/mat3ra/made/tools/build/grain_boundary/builders.py#L103)
`EDGE_INCLUSION_TOLERANCE` is a fine-tuning parameter that controls the inclusion of the edge atoms for both orientations in the gap.
For example of Graphene at 17.9 degrees: orange and green atoms are present with the value of 0.5 Angstroms, with value of 0, they will not be included.
<img src="https://i.imgur.com/QRgotXS.png" alt="Edge Inclusion Tolerance" width="400"/>


In [ ]:
# Material selection
MATERIAL_NAME = "Boron_Nitride"  # Name of the material to import from Standata

# Grain boundary parameters
TARGET_TWIST_ANGLE = 9.0  # in degrees
BOUNDARY_GAP = 0.0  # Gap between two orientations in X direction, in Angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 2]] # Supercell matrix to be applied to each of the orientations before matching

# Search algorithm parameters
MAX_REPETITION = None  # Maximum supercell matrix element value
ANGLE_TOLERANCE = 0.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Distance tolerance for two atoms to be considered too close. 
# Used when merging two orientations to remove the atoms of the first one. 
# Should be less than the expected bond length
DISTANCE_TOLERANCE = 1.43  # in Angstroms

# How much to expand inclusion of the edge atoms for both orientations and fill in the gap region.
# A fine-tuning parameter
EDGE_INCLUSION_TOLERANCE = 0.0  # in Angstroms

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
CELL_REPETITIONS_FOR_VISUALIZATION = [3, 3, 1]

### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install`.

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.3. Load and preview input material

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

material = Material(Materials.get_by_name_first_match(MATERIAL_NAME))

## 2. Prepare Material
### 2.1. Select and visualize initial material

In [ ]:
from utils.visualize import visualize_materials

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=CELL_REPETITIONS_FOR_VISUALIZATION)

## 3. Generate Surface Grain Boundary
### 3.1. Set up grain boundary configuration and builder


In [ ]:
from mat3ra.made.tools.build.grain_boundary import (
    SurfaceGrainBoundaryConfiguration,
    SurfaceGrainBoundaryBuilderParameters,
    SurfaceGrainBoundaryBuilder
)

config = SurfaceGrainBoundaryConfiguration(
    film=material,
    twist_angle=TARGET_TWIST_ANGLE,
    distance_z=BOUNDARY_GAP,
    gap=BOUNDARY_GAP,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX
)

params = SurfaceGrainBoundaryBuilderParameters(
    max_supercell_matrix_int=MAX_REPETITION,
    angle_tolerance=ANGLE_TOLERANCE,
    return_first_match=RETURN_FIRST_MATCH,
    edge_inclusion_tolerance=EDGE_INCLUSION_TOLERANCE,
    distance_tolerance=DISTANCE_TOLERANCE
)

builder = SurfaceGrainBoundaryBuilder(build_parameters=params)

### 3.2. Generate and analyze grain boundaries


In [ ]:
from utils.plot import plot_twisted_interface_solutions

grain_boundaries = builder.get_materials(config)

print(f"\nFound {len(grain_boundaries)} possible structures")
for i, gb in enumerate(grain_boundaries):
    actual_angle = gb.metadata.get("actual_twist_angle", "unknown")
    print(f"\nGrain Boundary {i + 1}:")
    print(f"Actual twist angle: {actual_angle}°")
    print(f"Number of atoms: {len(gb.basis.elements.ids)}")

if len(grain_boundaries) > 0:
    plot_twisted_interface_solutions(grain_boundaries)

## 4. Preview the selected grain boundary
By default, the first grain boundary is selected. You can change the selection by changing the `selected_structure` index.

In [ ]:
selected_structure = grain_boundaries[0]

actual_angle = selected_structure.metadata.get("build").get("configuration").get("actual_twist_angle")
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Actual angle: {actual_angle}°")
print(f"Number of atoms: {len(selected_structure.basis.elements.ids)}")

visualize_materials(selected_structure, repetitions=[1, 1, 1])
visualize_materials(selected_structure, repetitions=[1, 1, 1], rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(selected_structure)
download_content_to_file(selected_structure, "grain_boundary_2d_boron_nitride.json")